In [2]:
from __future__ import unicode_literals
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import date as dt, timedelta
from typing import List
from google.cloud import firestore
from functools import reduce
import time
import base64
import sys
import math


In [3]:
%env GOOGLE_APPLICATION_CREDENTIALS="/Users/marcelcanhisares/Development/tcc/keys/mc-tcc1-2893283b8dce.json"

env: GOOGLE_APPLICATION_CREDENTIALS="/Users/marcelcanhisares/Development/tcc/keys/mc-tcc1-2893283b8dce.json"


In [4]:
%ls /Users/marcelcanhisares/Development/tcc/keys/

mc-tcc1-2893283b8dce.json


In [5]:
DB = firestore.Client.from_service_account_json("/Users/marcelcanhisares/Development/tcc/keys/mc-tcc1-2893283b8dce.json")

In [6]:
collection = DB.collection('analytics').stream()

In [7]:
resultDicts = []
for document in collection:
    resultDicts.append(document.to_dict())
    

In [13]:
resultDicts[0]['date']

'2020-04-23'

In [9]:
resultlist = []
for result in resultDicts:
    if result['date'] == '2020-04-23' or result['date'] == '2020-04-24' or result['date'] == '2020-04-30': continue
    resultlist.append({'Resultados': result['totalResults'], 'Colisões': result['totalCollisions'], 'Data': result['date'] })
df = pd.DataFrame(resultlist)
df['Data'] = pd.to_datetime(df['Data'])

In [10]:
df.set_index('Data', inplace=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 141 entries, 2020-05-01 to 2020-10-13
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Resultados  141 non-null    int64
 1   Colisões    141 non-null    int64
dtypes: int64(2)
memory usage: 3.3 KB


In [11]:
df

,Resultados,Colisões
Data,,
2020-05-01,1338,302
2020-05-02,679,312
2020-05-03,410,199
2020-05-04,273,81
2020-05-05,1194,385
...,...,...
2020-10-11,416,115
2020-10-12,214,33
2020-10-13,374,121


In [11]:
temp = df

In [12]:
temp = temp.resample('M').sum()

In [13]:
temp

,Resultados,Colisões
Data,,
2020-05-31,40598,12464
2020-06-30,35493,12274
2020-07-31,56493,20732
2020-08-31,56827,20787
2020-09-30,38539,12445
2020-10-31,13121,4158


In [14]:
import matplotlib.dates as mdates

In [15]:
temp.index

DatetimeIndex(['2020-05-31', '2020-06-30', '2020-07-31', '2020-08-31',
               '2020-09-30', '2020-10-31'],
              dtype='datetime64[ns]', name='Data', freq='M')

In [16]:
def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

In [17]:
%matplotlib tk

In [18]:
# %matplotlib inline

fig, ax = plt.subplots(figsize=(15,10))
width = 12
shift = pd.to_timedelta('12D')
results = ax.bar(temp.index, temp["Resultados"], width=width, label='Resultados')
colisions = ax.bar(temp.index+shift, temp["Colisões"], width=width, label='Colisões')
ax.legend()
ax.set_xticks(temp.index)
ax.set_title("Tweets coletados por data")
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
ax.xaxis.set_minor_formatter(mdates.DateFormatter("%Y-%m"))
autolabel(results)
autolabel(colisions)
fig.tight_layout()

plt.show()



In [20]:
df.sum()

Resultados    241071
Colisões       82860
dtype: int64

In [21]:
dfs = []
for result in resultDicts:
    t_df = pd.DataFrame(result['results'].items())
    t_df['date'] = result['date']
    t_df.rename(columns={ t_df.columns[0]: "STOCK" }, inplace = True)
    t_df = t_df.pivot(index ='date', columns ='STOCK')
    dfs.append(t_df)

In [22]:
df = pd.concat(dfs)

In [23]:
df.columns = df.columns.get_level_values(1)

In [24]:
pd.options.display.max_rows = 4000

In [25]:
sum_df = df.select_dtypes(np.number).sum()

In [26]:
sum_df.sum()

169174.0

In [19]:
sum_df.nlargest(5)

STOCK
OIBR3    14984.0
PETR4     9518.0
MGLU3     6574.0
BBDC4     6150.0
JHSF3     5769.0
dtype: float64

In [27]:
largest = sum_df.nlargest(5)

In [28]:
%matplotlib tk

In [29]:
sum_df = sum_df.rename_axis('Ativo')

In [31]:
my_colors = [(x/50, x/100, 0.75) for x in range(len(sum_df.nlargest(10)))]
fig = sum_df.nlargest(10).plot.bar(rot=90, y='Count', x='Ativo', figsize=(10,5), color=my_colors, title="Menções por ativo", fontsize=10)
fig.set_title('Menções por ativo', fontdict = {'fontsize' : 15})
fig.set_xlabel('Ativo', fontsize=15)
fig.set_ylabel('Menções', fontsize=15)

Text(47.5, 0.5, 'Menções')

In [32]:
fig.plot()

[]

In [24]:
sum_df.nlargest(30)

Ativo
OIBR3     14984.0
PETR4      9518.0
MGLU3      6574.0
BBDC4      6150.0
JHSF3      5769.0
VALE3      5647.0
BBAS3      4979.0
MRFG3      4571.0
CIEL3      4149.0
ITUB4      4049.0
BEEF3      3964.0
AZUL4      3710.0
ITSA4      3710.0
EVEN3      3193.0
WEGE3      3070.0
EMBR3      2939.0
JBSS3      2580.0
GOLL4      2467.0
SHOW3      2333.0
CVCB3      2052.0
BTOW3      2016.0
OIBR4      1926.0
PETR3      1863.0
RAPT3      1790.0
RENT3      1760.0
CSNA3      1606.0
ABEV3      1582.0
GGBR4      1571.0
BOVA11     1524.0
ELET3      1512.0
dtype: float64

In [25]:
collection = DB.collection('tweets').stream()

In [26]:
count = 0
date_dict = dict()
dates = []
for tweet in collection:
    date = tweet.to_dict()['date'].strftime('%Y-%m-%d')
    date_dict[date] = date_dict[date] + 1 if date in date_dict else 1
    count = count + 1

DeadlineExceeded: 504 Deadline Exceeded

In [ ]:
date_dict